In [41]:
import joblib

# Modeli kaydetme
model_filename = r"..\Model\random_forest_model.joblib"

# Modeli yükleme
loaded_model = joblib.load(model_filename)
print("Model başarıyla yüklendi.")


Model başarıyla yüklendi.


In [70]:
import joblib
import librosa
import numpy as np

# Scaler ve model dosyalarını yükleme
scaler_filename = '..\\Model\\scaler.joblib'  # Modelin scaler dosyasının yolu
model_filename = '..\\Model\\random_forest_model.joblib'  # Model dosyasının yolu

scaler = joblib.load(scaler_filename)
model = joblib.load(model_filename)

def extract_features_for_prediction(file_path):
    """Ses dosyasından özellikleri çıkar."""
    try:
        # Ses dosyasını yükle
        y, sr = librosa.load(file_path, sr=None)
        
        # MFCC (Mel-frequency cepstral coefficients) çıkarımı
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)

        # Chroma çıkarımı (müzikal tonlar)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_mean = np.mean(chroma, axis=1)

        # RMS Energy çıkarımı
        rms = librosa.feature.rms(y=y)
        rms_mean = np.mean(rms)

        # Zero-Crossing Rate çıkarımı
        zcr = librosa.feature.zero_crossing_rate(y=y)
        zcr_mean = np.mean(zcr)
        
        # Özellikleri birleştir
        features = np.hstack((mfcc_mean, chroma_mean, rms_mean, zcr_mean))
        return features

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def predict_audio_with_scaler(file_path):
    """Ses dosyasını scaler ve model ile test et."""
    # Özellik çıkarma
    features = extract_features_for_prediction(file_path)
    if features is None:
        print("Özellik çıkarma başarısız.")
        return

    # Özellikleri normalize et
    features = features.reshape(1, -1)  # Modelin tahmin yapabilmesi için 2D array haline getir
    normalized_features = scaler.transform(features)

    # Model ile tahmin yap
    prediction = model.predict(normalized_features)
    print(f"Tahmin edilen etiket: {prediction[0]}")

# Test edilecek ses dosyası
test_file = "C:\\VoiceProject\\Voice\\irem\\korku\\Iremkorku3.wav"  # Test ses dosyasının tam yolunu yazın
predict_audio_with_scaler(test_file)


Tahmin edilen etiket: irem


c:\Users\iremm\.conda\envs\BTK-AI\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [ ]:
import sounddevice as sd
import numpy as np
import librosa
import joblib
from sklearn.preprocessing import MinMaxScaler

# Özellikleri çıkarma (MFCC, Chroma, RMS, ZCR)
def extract_features_from_audio(audio_data, sr=16000):
    try:
        # MFCC çıkarımı
        mfcc = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)  # MFCC ortalamasını al

        # Chroma çıkarımı
        chroma = librosa.feature.chroma_stft(y=audio_data, sr=sr)
        chroma_mean = np.mean(chroma, axis=1)

        # RMS Energy çıkarımı
        rms = librosa.feature.rms(y=audio_data)
        rms_mean = np.mean(rms)

        # Zero-Crossing Rate çıkarımı
        zcr = librosa.feature.zero_crossing_rate(y=audio_data)
        zcr_mean = np.mean(zcr)
        
        # Özellikleri birleştir
        features = np.hstack((mfcc_mean, chroma_mean, rms_mean, zcr_mean))
        return features
    
    except Exception as e:
        print(f"Error processing audio: {e}")
        return None

# Mikrofondan ses kaydetme
def record_audio(duration=5, fs=16000):
    print("Ses kaydediliyor...")
    audio_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()  # Kaydın tamamlanmasını bekleyin
    print("Ses kaydı tamamlandı.")
    return audio_data.flatten()  # Ses verisini düzleştiriyoruz (1D array)

# Ses kaydını işleyip tahmin yapma
def process_audio_file(audio_data, scaler, model):
    # 1. Adım: Veriyi 0-1 arası normalizasyon
    audio_data = (audio_data - np.min(audio_data)) / (np.max(audio_data) - np.min(audio_data))  # 0-1 arası

    # 2. Adım: Özellik çıkarımı
    features = extract_features_from_audio(audio_data)

    if features is not None:
        # 3. Adım: Özellikleri normalize etme
        normalized_features = scaler.transform([features])
        print(f"Normalized Features: {normalized_features}")

        # 4. Adım: Model ile tahmin yapma
        prediction = model.predict(normalized_features)
        print(f"Tahmin edilen sınıf: {prediction[0]}")
    else:
        print("Özellik çıkarımı yapılamadı, lütfen sesi yeniden kaydedin.")

# Model ve scaler'ı yükleyin
model = joblib.load('..\Model\\random_forest_model.joblib')
scaler = joblib.load('..\Model\\scaler.joblib')

# Mikrofondan ses kaydedin (5 saniye)
audio_data = record_audio(duration=5)

# Kaydedilen sesi işleyip tahmin yapma
process_audio_file(audio_data, scaler, model)


Ses kaydediliyor...
Ses kaydı tamamlandı.
Normalized Features: [[ 1.4385326   0.41547256  0.14244706  0.58976865  0.60860644  1.06585869
   0.81974664  0.7084436   0.88331646  0.78442708  0.46620559  0.70392766
   0.85344995  0.52587329  0.31204448  0.23396043  0.18220832  0.0877071
   0.0553659   0.05551352  0.07892574  0.16027521  0.5353755   1.18515853
   0.69448006  4.81318403 -0.07128268]]
Tahmin edilen sınıf: irem


c:\Users\iremm\.conda\envs\BTK-AI\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


: 